Developed by Chrisowalandis Deligio, Sebastian Gampe

In [1]:
import sys
sys.path.append('../')
import pandas as pd
import random
import os
import numpy as np
from cnt.generate_re_data import AutoRE

from cnt.annotate import (annotate, annotate_single_design, 
                          annotate_designs, 
                          extract_string_from_annotation, labeling_eng)
from cnt.extract_relation import (path, NERTransformer, FeatureExtractor)
from cnt.evaluate import Metrics
from cnt.vectorize import (Doc2Str, Path2Str, Verbs2Str, AveragedPath2Vec, 
                           AveragedRest2Vec)
from cnt.io import (replace_left_right)
from cnt.io import  Database_Connection
from cnt.preprocess import Preprocess

import warnings
warnings.filterwarnings('ignore')

In [2]:
auto = AutoRE()

In [3]:
subjects = ["Athena", "Ares", "Apollo", "Zeus", "Nike", "Olybrius", "Seleucus I", "Domitianus", "Equitas", "Rhoemetalces I"]

In [4]:
relations = {
    "holding":["sword", "shield", "paludamentum", "staff", "club", "stones", "jar", "bow"],
    "wearing":["belt", "crown", "helmet", "cuirass", "crown juwel", "ribbons", "boots"],
    "seated_on":["dolphin", "shield", "omphalos", "rock", "bull", "panther", "kline"],
    "resting_on":["serpent staff", "torch", "shield", "panther", "club", "water urn", "knee", "rock"],
    "feeding":["panther", "dolphin"],
    "receiving":["apple", "agonistic crown", "crown", "shield"],}

In [5]:
auto.set_subjects(subjects)

In [6]:
auto.set_relations(relations)

In [7]:
result = auto.generate(300)

In [8]:
result.head(5)

,design_en,y
0,Rhoemetalces I wearing helmet.,"(Rhoemetalces I, wearing, helmet)"
1,Ares seated on panther.,"(Ares, seated_on, panther)"
2,Olybrius receiving shield.,"(Olybrius, receiving, shield)"
3,Athena receiving crown.,"(Athena, receiving, crown)"
4,Domitianus receiving apple.,"(Domitianus, receiving, apple)"


In [9]:
X = result[["design_en"]]
X["design_de"] = ""
X["comment"] = "auto_generated"
X["design_en_changed"] = ""
X.head(2)

,design_en,design_de,comment,design_en_changed
0,Rhoemetalces I wearing helmet.,,auto_generated,
1,Ares seated on panther.,,auto_generated,


In [10]:
X.shape

(300, 4)

In [11]:
# Database access in text file: "mysql+mysqlconnector://root:YourConnection" -> Format user:password@IP/Database
f = open("/home/bigdatalab/Projects/D4N4/NLP_release_1.0/db_access.txt", "r")
access = f.read()
dc =  Database_Connection(access)

In [12]:
X.to_sql("nlp_training_designs",dc.mysql_connection,if_exists="append", index=False)

300

In [13]:
def get_id(design):
    try:
        return (dc.create_own_query("select id from nlp_training_designs where design_en='"+design+"';").id)[0]
    except:
        return "Null"

In [14]:
X["db_id"] = X.apply(lambda row: get_id(row.design_en), axis=1)

In [15]:
X.head(5)

,design_en,design_de,comment,design_en_changed,db_id
0,Rhoemetalces I wearing helmet.,,auto_generated,,27705
1,Ares seated on panther.,,auto_generated,,27760
2,Olybrius receiving shield.,,auto_generated,,27593
3,Athena receiving crown.,,auto_generated,,27590
4,Domitianus receiving apple.,,auto_generated,,27784


In [16]:
X = X.loc[X.db_id!="Null"]
X.shape

(300, 5)

In [17]:
X["y"] = result["y"]

In [18]:
X.head(5)

,design_en,design_de,comment,design_en_changed,db_id,y
0,Rhoemetalces I wearing helmet.,,auto_generated,,27705,"(Rhoemetalces I, wearing, helmet)"
1,Ares seated on panther.,,auto_generated,,27760,"(Ares, seated_on, panther)"
2,Olybrius receiving shield.,,auto_generated,,27593,"(Olybrius, receiving, shield)"
3,Athena receiving crown.,,auto_generated,,27590,"(Athena, receiving, crown)"
4,Domitianus receiving apple.,,auto_generated,,27784,"(Domitianus, receiving, apple)"


## Add relations to db

In [19]:
entities = dc.create_own_query("select * from nlp_list_entities;")
entities[["id", "name_en", "alternativenames_en"]]

,id,name_en,alternativenames_en
0,1,Abundantia,None
1,2,Actaeon,None
2,3,Aemilian,None
3,4,Aeneas,None
4,5,Aequitas,Equitas
...,...,...,...
844,845,turning,None
845,846,sailing,None
846,847,escorted_by,escorted by
847,848,wearing,None


In [20]:
X["y"] = X.apply(lambda row: [row.y], axis=1)

In [21]:
ent_dict = {}
for index, row in entities[["id", "name_en", "alternativenames_en"]].iterrows():
    ent_dict[row["id"]] = (row["name_en"]+", "+row["alternativenames_en"]).replace(", None","").lower()

In [22]:
result = pd.DataFrame(columns=["design_id", "subject", "predicate", "object"])

In [23]:
def get_id(ent):
    for row in ent_dict:
        for entry in ent_dict[row].split(', '):
            if ent == entry:
                return row
            if ent.lower() == entry:
                return row
    return ent

In [24]:
for index, row in X.iterrows():
    for y in row.y:
        subject = get_id(y[0])
        subject_str = y[0]
        predicate = get_id(y[1])
        predicate_str = y[1]
        obj = get_id(y[2])
        obj_str = y[2]
        if type(subject) == int and type(predicate) == int and type(obj) == int:
            result = result._append({"design_id": row.db_id, 
                                     "subject": subject, 
                                     "subject_str": subject_str,
                                     "predicate": predicate, 
                                     "predicate_str": predicate_str,
                                     "object": obj,
                                     "object_str": obj_str}, ignore_index=True)
        else:
            print(subject, predicate, obj)

In [25]:
result.shape

(300, 7)

In [27]:
result.style

,design_id,subject,predicate,object,object_str,predicate_str,subject_str
0,27705,322,848,524,helmet,wearing,Rhoemetalces I
1,27760,34,827,763,panther,seated_on,Ares
2,27593,268,836,612,shield,receiving,Olybrius
3,27590,41,836,489,crown,receiving,Athena
4,27784,114,836,792,apple,receiving,Domitianus
5,27744,114,848,524,helmet,wearing,Domitianus
6,28170,5,828,650,torch,resting_on,Equitas
7,28278,41,828,612,shield,resting_on,Athena
8,27762,268,828,650,torch,resting_on,Olybrius
9,27809,322,827,535,kline,seated_on,Rhoemetalces I


In [28]:
result.to_sql("nlp_relation_extraction_en_v3",dc.mysql_connection,if_exists="append", index=False)

300